In [30]:
from metar import Metar
from datetime import datetime, timedelta
from metar import Metar
from util.utils import *
import pandas as pd
import re


In [31]:

def interpretaMetaR(metarString):

    if("METAF" in metarString):
        metarString = metarString.replace('METAF', 'METAR')
        metarString = metarString.replace('\n', '')


    station = None
    type = None
    hora_formated = None
    temperature = None
    dew_point = None
    windDirection = None
    windVelocity = None
    visibility = None
    pressure = None
    weather = None
    sky = None
    skyFeet= None

    try: 

        obs = Metar.Metar(metarString[:-1])

        for item in obs.string().split('\n'):
            if("station" in item):
                station = item.split(" ")[1]
            if("type" in item):
                type = " ".join(item.split(" ")[1:])
            if("time" in item):
                timeZ = " ".join(item.split(" ")[1:])
                data_obj = datetime.strptime(timeZ, "%a %b %d %H:%M:%S %Y")
                nova_data_obj = data_obj - timedelta(hours=3)
                hora_formated = nova_data_obj.strftime("%a %b %d %H:%M:%S %Y")
            if("temperature" in item):
                temperature = item.split(" ")[1]
            if("dew point" in item):
                dew_point = item.split(" ")[2]
            if("wind" in item):
                wind = (item.split(" ")[1:])
                windDirection = wind[0]
                windVelocity = wind[2]
            if("visibility" in item):
                visibility = item.split(" ")[1]
            if("pressure" in item):
                pressure = item.split(" ")[1]
            if("weather" in item):
                weather = " ".join(item.split(" ")[1:])
            if("sky" in item):
                skyString = " ".join(item.split(" ")[1:])
                if(' at ' in skyString):
                    sky = ''.join(skyString.split(' at ')[0])
                else:
                    sky = ''.join(skyString.split(',')[0])
                
                allFeets = re.findall(r"(\d+)",skyString)
                if len(allFeets) > 0:
                    skyFeet = allFeets[0]
    except:
        pass

    return station, type, hora_formated, temperature,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet

def direction_to_degrees(df):
    direction_degrees = {
    'N': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'E': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'S': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'W': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5,
    'calm': 0,
    'variable': 0,
    }
    try:
        df['metar_windDirection'] = df['metar_windDirection'].map(direction_degrees)
    except:
        df['metaf_windDirection'] = df['metaf_windDirection'].map(direction_degrees)

    return df

def defineIndex(df):
    df = df.sort_values(by='hora_formatted')
    df.set_index('hora_formatted', inplace=True)
    df.index = pd.to_datetime(df.index, format='%A, %B %d, %Y %H:%M:%S')
    return df

def to_dummies(df):
    df_Dummies = pd.get_dummies(df['sky'])
    df_final = pd.concat([df, df_Dummies], axis=1)

    return df_final

def sort_df(df):
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.sort_values(by=['aero','hora_formatted'])

    return df
def calculate_average(series):
    result = []
    for i in range(len(series) - 1):
        current_value = series.iloc[i]
        next_value = series.iloc[i + 1]
        average = (current_value + next_value) / 2
        result.append(average)
    return pd.Series(result, index=series.index[:-1])

def pipemediaTempoMetar(df, colunas):
    listSeries = []
    non_numeric_columns = []

    for coluna in colunas:
        if pd.api.types.is_numeric_dtype(df[coluna]):
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
            df_resampled_temp = df[coluna].resample('30T').mean()
            df_resampled_temp_filled = df_resampled_temp.fillna(method='ffill')
            tempSeries = calculate_average(df_resampled_temp_filled)
            listSeries.append(tempSeries)
        else:
            non_numeric_columns.append(coluna)

    dataframe_resultante = pd.concat(listSeries, axis=1)

    # Criar um novo DataFrame para armazenar as colunas não numéricas
    df_nao_numericas = df[non_numeric_columns]

    # Redefinir os índices dos DataFrames antes da concatenação
    dataframe_resultante.reset_index(drop=True, inplace=True)
    df_nao_numericas.reset_index(drop=True, inplace=True)

    # Concatenar o DataFrame resultante com o DataFrame das colunas não numéricas
    resultado_final = pd.concat([dataframe_resultante, df_nao_numericas], axis=1)

    return resultado_final

def read_metar(df,tabela):
    if (tabela == "metar"):
        resultado_df =  df['metar'].apply(interpretaMetaR)
        resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['metar_station', 'metar_type', 'metar_hora_formated', 'metar_station','metar_dew_point','metar_windDirection','metar_windVelocity','metar_visibility','metar_pressure','metar_weather','metar_sky','metar_skyFeet'] )
    else:
        resultado_df =  df['metaf'].apply(interpretaMetaR)
        resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['metaf_station', 'metaf_type', 'metaf_hora_formated', 'metaf_station','metaf_dew_point','metaf_windDirection','metaf_windVelocity','metaf_visibility','metaf_pressure','metaf_weather','metaf_sky','metaf_skyFeet'] )
    df_bronze = pd.concat([df, resultado_df], axis=1)
    df_bronze = direction_to_degrees(df_bronze)
    # df_bronze = sort_df(df_bronze)
    # df_bronze = df_bronze.reset_index(drop=True)
    # df_bronze['hora_formatted'] = pd.to_datetime(df_bronze['hora_formatted'])
    # df_bronze.set_index(['aero','hora_formatted'], inplace=True)
    return df_bronze
    df_media = pipemediaTempoMetar(df_bronze,['hora','aero','dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'])
    df_no_columns = df_bronze.drop(['dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'],axis=1)
    return df_media,df_bronze



In [32]:
df = pd.read_csv("../data/processed_data/merged_bronze.csv")

In [33]:
df = read_metar(df,"metar")
df = read_metar(df,"metaf")

In [34]:
df.columns

Index(['Unnamed: 0', 'flightid', 'origem', 'destino', 'dt_dep', 'dt_arr',
       'dt_dep_formatted', 'dt_arr_formatted', 'hora_real', 'nova_cabeceira',
       'antiga_cabeceira', 'aero_real', 'hora_formatted_real', 'hora_prev',
       'troca', 'aero_prev', 'hora_formatted_prev', 'hora_metar', 'metar',
       'aero_metar', 'hora_formatted_metar', 'hora_metaf', 'metaf',
       'aero_metaf', 'hora_formatted_metaf', 'metar_station', 'metar_type',
       'metar_hora_formated', 'metar_station', 'metar_dew_point',
       'metar_windDirection', 'metar_windVelocity', 'metar_visibility',
       'metar_pressure', 'metar_weather', 'metar_sky', 'metar_skyFeet',
       'metaf_station', 'metaf_type', 'metaf_hora_formated', 'metaf_station',
       'metaf_dew_point', 'metaf_windDirection', 'metaf_windVelocity',
       'metaf_visibility', 'metaf_pressure', 'metaf_weather', 'metaf_sky',
       'metaf_skyFeet'],
      dtype='object')

In [35]:
df[['flightid','dt_dep_formatted', 'dt_arr_formatted','hora_formatted_metar','hora_formatted_metaf']]

,flightid,dt_dep_formatted,dt_arr_formatted,hora_formatted_metar,hora_formatted_metaf
0,810f965ce283a33ab6cbb303ccaaaddd,"Friday, March 31, 2023 17:51:28","Friday, March 31, 2023 19:07:26","Friday, March 31, 2023 18:00:00","Friday, March 31, 2023 19:00:00"
1,d1e8b77083f797ad9f69ebf4d2df0e13,"Wednesday, March 29, 2023 18:50:18","Wednesday, March 29, 2023 20:00:43","Wednesday, March 29, 2023 19:00:00","Wednesday, March 29, 2023 20:00:00"
2,5b515ea8e42b84f9f3aa57fdd28d6937,"Monday, March 27, 2023 05:51:40","Monday, March 27, 2023 07:55:54","Monday, March 27, 2023 06:00:00","Monday, March 27, 2023 08:00:00"
3,558f95b81bae1856ba3d589bd0218de5,"Friday, March 31, 2023 14:25:16","Friday, March 31, 2023 15:57:21","Friday, March 31, 2023 14:25:00","Friday, March 31, 2023 16:00:00"
4,558f95b81bae1856ba3d589bd0218de5,"Friday, March 31, 2023 14:25:16","Friday, March 31, 2023 15:57:21","Friday, March 31, 2023 14:25:00","Friday, March 31, 2023 16:00:00"
...,...,...,...,...,...
827,02c8b7a6105b2ca6575ea32cc80df80d,"Friday, March 31, 2023 21:06:57","Friday, March 31, 2023 21:56:59","Friday, March 31, 2023 21:00:00","Friday, March 31, 2023 22:00:00"
828,ec3d181fe2600893412145fdb47b32e0,"Thursday, March 09, 2023 09:55:27","Thursday, March 09, 2023 11:08:39","Thursday, March 09, 2023 10:00:00","Thursday, March 09, 2023 11:00:00"
829,ec3d181fe2600893412145fdb47b32e0,"Thursday, March 09, 2023 09:55:27","Thursday, March 09, 2023 11:08:39","Thursday, March 09, 2023 10:00:00","Thursday, March 09, 2023 11:00:00"
830,ec3d181fe2600893412145fdb47b32e0,"Thursday, March 09, 2023 09:55:27","Thursday, March 09, 2023 11:08:39","Thursday, March 09, 2023 10:00:00","Thursday, March 09, 2023 11:00:00"


In [37]:
df = df.drop_duplicates()

df.shape

(832, 49)

In [36]:
df.to_csv("../data/processed_data/merged_plate.csv")